In [1]:
# bwrgb_v1_viz / BW2RGB
# Gourav Siddhad
# 10-November-2019

In [2]:
from __future__ import absolute_import, division, print_function

print('Importing Libraries', end='')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from numpy.random import seed
seed(1607)
from tensorflow import set_random_seed
set_random_seed(1607)

from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.layers import Input, Dense, Layer, Reshape, ReLU, Add
from keras.layers.core import Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
import keras.backend as K
from skimage.io import imsave

import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

from sklearn.model_selection import train_test_split
from skimage.color import rgb2lab, lab2rgb

print('  - Done')

Importing Libraries

C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

  - Done


In [3]:
from keras.models import model_from_json

json_file = open('bw2rgb.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.load_weights('bw2rgb.h5')
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()









_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv_0 (Sequential)          (None, 64, 64, 64)        112448    
_________________________________________________________________
conv_1 (Sequential)          (None, 32, 32, 128)       518656    
_________________________________________________________________
conv_2 (Sequential)          (None, 16, 16, 256)       2069504   
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 128, 4)       0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 128, 128, 2)       74        
Total params: 2,700,682
Trainable params: 2,697,098
Non-trainable params: 3,584
______________________________________________________

In [4]:
H, W, Ci, Co = 128, 128, 1, 2

db = 'C:/Users/Gourav Siddhad/Documents/BW2RGB/Project/'
db1 = 'test_128/'
db2 = 'test_large/'
db3 = 'test_under/'

In [5]:
dbt = db + db1
print(dbt)
images = []
imlist = sorted(os.listdir(dbt))

# Load Images
for i, im in enumerate(imlist):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    img = cv2.cvtColor(cv2.imread(dbt + im), cv2.COLOR_BGR2RGB)
    img_lab = rgb2lab(img)
    img_l = img_lab[:, :, 0]/100.0
    images.append(img_l)

images = np.array(images)
print()
print(images.shape)

# Convert Images
for i, im in enumerate(images):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    t_ab = model.predict(im.reshape(1, H, W, Ci))
    t_ab = t_ab.reshape(H, W, Co)

    temp = np.zeros((H, W, Ci+Co))
    temp[:, :, 0] = im*100.0
    temp[:, :, 1:] = t_ab*128.0
    temp = lab2rgb(temp.reshape(H, W, Ci+Co))
    temp = np.array(temp*255, dtype='uint8')
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(db1 + imlist[i], temp)

C:/Users/Gourav Siddhad/Documents/BW2RGB/Project/test_128/
....................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
(500, 128, 128)
.........

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


....................

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 7 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


...............................................

...

..................................

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 12 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


.


C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 15 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


...................................................................................................

In [6]:
dbt = db + db2
print(dbt)
images = []
imlist = sorted(os.listdir(dbt))

# Load Images
for i, im in enumerate(imlist):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    img = cv2.cvtColor(cv2.imread(dbt + im), cv2.COLOR_BGR2RGB)
    img_lab = rgb2lab(img)
    img_l = img_lab[:, :, 0]/100.0
    images.append(img_l)

images = np.array(images)
print()
print(images.shape)

# Convert Images
for i, im in enumerate(images):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    if float(im.shape[0]//H)==im.shape[0]/H:
        a = im.shape[0]
    else:
        a = H*(im.shape[0]//H) + H
        
    if float(im.shape[1]//W)==im.shape[1]/W:
        b = im.shape[1]
    else:
        b = W*(im.shape[1]//W) + W
        
    pimg = np.zeros((a, b))
    pimg[0:im.shape[0], 0:im.shape[1]] = im
        
    new_img = np.zeros((a, b, Ci+Co))
    for j in range(pimg.shape[0]//H):
        for k in range(pimg.shape[1]//W):
            t_l = pimg[j*H:(j+1)*H, k*W:(k+1)*W]

            t_ab = model.predict(t_l.reshape(1, H, W, Ci))
            t_ab = t_ab.reshape(H, W, Co)

            temp = np.zeros((H, W, Ci+Co))
            temp[:, :, 0] = t_l*100.0
            temp[:, :, 1:] = t_ab*128.0
            temp = lab2rgb(temp)

            new_img[j*H:(j+1)*H, k*W:(k+1)*W, :] = temp.reshape(H, W, Ci+Co)
    
    newimg = np.array(new_img*255, dtype='uint8')
    newimg = newimg[0:im.shape[0], 0:im.shape[1]]
    newimg = cv2.cvtColor(newimg, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(db2 + imlist[i], newimg)

C:/Users/Gourav Siddhad/Documents/BW2RGB/Project/test_large/
..
(2,)
.

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 121 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 63 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 90 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 16 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 67 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\

.

In [7]:
dbt = db + db3
print(dbt)
images = []
imlist = sorted(os.listdir(dbt))

# Load Images
for i, im in enumerate(imlist):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    img = cv2.cvtColor(cv2.imread(dbt + im), cv2.COLOR_BGR2RGB)
    img_lab = rgb2lab(img)
    img_l = img_lab[:, :, 0]/100.0
    images.append(img_l)

images = np.array(images)
print()
print(images.shape)

# Convert Images
for i, im in enumerate(images):
    if i%100==0 and i!=0:
        print()
    else:
        print('.', end='')
        
    if float(im.shape[0]//H)==im.shape[0]/H:
        a = im.shape[0]
    else:
        a = H*(im.shape[0]//H) + H
        
    if float(im.shape[1]//W)==im.shape[1]/W:
        b = im.shape[1]
    else:
        b = W*(im.shape[1]//W) + W
        
    pimg = np.zeros((a, b))
    pimg[0:im.shape[0], 0:im.shape[1]] = im
        
    new_img = np.zeros((a, b, Ci+Co))
    for j in range(pimg.shape[0]//H):
        for k in range(pimg.shape[1]//W):
            t_l = pimg[j*H:(j+1)*H, k*W:(k+1)*W]

            t_ab = model.predict(t_l.reshape(1, H, W, Ci))
            t_ab = t_ab.reshape(H, W, Co)

            temp = np.zeros((H, W, Ci+Co))
            temp[:, :, 0] = t_l*100.0
            temp[:, :, 1:] = t_ab*128.0
            temp = lab2rgb(temp)

            new_img[j*H:(j+1)*H, k*W:(k+1)*W, :] = temp.reshape(H, W, Ci+Co)
    
    newimg = np.array(new_img*255, dtype='uint8')
    newimg = newimg[0:im.shape[0], 0:im.shape[1]]
    newimg = cv2.cvtColor(newimg, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(db3 + imlist[i], newimg)

C:/Users/Gourav Siddhad/Documents/BW2RGB/Project/test_under/
.......
(7,)
..

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 80 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


...

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 304 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


.

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 33 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 509 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 418 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 396 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


.

C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 600 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 10 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 39 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 104 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 54 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Anaconda\lib\site-packages\skimage\color